In [1]:
import Functions as F 
import pandas as pd 
import numpy as np 
%matplotlib qt 
results = "results/results03.csv"
nr_agents = 6
nr = 14
Viz = F.VisualizeAngle(results, nr_agents)
Viz.get_coor()
Viz.plot_angle_in_plane(nr, True)

rotation_ann = Viz.R_ann
rotation_model = Viz.R_model 
Dij = F.Dijkstras(results, nr_agents, nr, rotation_ann, rotation_model)

Dij.run_dijkstra(6, 6)

In [2]:
Dij.plot_both()

## Plot path on top of image

In [35]:
import SimpleITK as sitk 
import numpy as np 
import os 
import matplotlib.pyplot as plt 

In [36]:
facial_paths = os.listdir("paths/paths_facial")
itkimage = sitk.ReadImage("Cmarl/src/data/images/Data_good_Scaled/" + facial_paths[0][6:])

mask = sitk.GetArrayFromImage(sitk.ReadImage("paths/paths_facial/" + facial_paths[0]))
im = sitk.GetArrayFromImage(itkimage)


In [38]:
import statistics
x, y, z = np.where(mask == 1)
slice = statistics.mode(y)

plt.imshow(im[:, slice], cmap = "gray")
plt.scatter(z[y == slice], x[y == slice], color ="r", s = 2)
plt.axis("off")
plt.show()

In [31]:
facial_paths[0][6:]

'008_1_R.nii.gz'